In [27]:
import requests
from bs4 import BeautifulSoup
import time
from fake_useragent import UserAgent

In [32]:
import requests
from bs4 import BeautifulSoup
import time

# Define the URL
url = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=14450;type=tournament'

# Define a static user agent
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# Send a GET request to the URL
for _ in range(5):  # Try 5 times
    try:
        # Send the request with the defined user agent
        response = requests.get(url, headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            break  # Break the loop if successful
        else:
            print(f"Failed to fetch URL. Status code: {response.status_code}")
            time.sleep(5)  # Wait for 5 seconds before retrying
    except Exception as e:
        print(f"Error: {e}")
        time.sleep(5)  # Wait for 5 seconds before retrying

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Now you can continue with your scraping logic


In [40]:
links = []
all_rows = soup.select('ds-w-full > tbody > tr')
for row in all_rows:
    row_url = "https://www.espncricinfo.com" + row.select('td:nth-child(7) a')[0]['href']
    links.append(row_url)

match_summary_links = links

In [41]:
match_summary_links

[]

In [3]:
match_details = []
for link in match_summary_links:
    response = requests.get(link)
    match_soup = BeautifulSoup(response.content, 'html.parser')

    match_info = match_soup.find('div', text='Match Details').parent.find_next_siblings()[0].text.strip()
    team1, team2 = match_info.split(' Vs ')

    tables = match_soup.select('div > table.ci-scorecard-table')
    first_innings_rows = tables[0].select('tbody > tr')
    second_innings_rows = tables[1].select('tbody > tr')

    for index, row in enumerate(first_innings_rows):
        tds = row.select('td')
        batting_summary.append({
            "match": match_info,
            "teamInnings": team1,
            "battingPos": index+1,
            "batsmanName": tds[0].select('a > span > span')[0].text.strip().replace(' ', ''),
            "dismissal": tds[1].select('span > span')[0].text.strip(),
            "runs": tds[2].select('strong')[0].text.strip(),
            "balls": tds[3].text.strip(),
            "4s": tds[5].text.strip(),
            "6s": tds[6].text.strip(),
            "SR": tds[7].text.strip()
        )
        
   for index, row in enumerate(second_innings_rows):
        tds = row.select('td')
        batting_summary.append({
            "match": match_info,
            "teamInnings": team2,
            "battingPos": index+1,
            "batsmanName": tds[0].select('a > span > span')[0].text.strip().replace(' ', ''),
            "dismissal": tds[1].select('span > span')[0].text.strip(),
            "runs": tds[2].select('strong')[0].text.strip(),
            "balls": tds[3].text.strip(),
            "4s": tds[5].text.strip(),
            "6s": tds[6].text.strip(),
            "SR": tds[7].text.strip()
        })

#------- 2.b Parser Code ------------#
batting_summary = match_details

# Return the result
print(batting_summary)


In [7]:
import requests
from bs4 import BeautifulSoup

# Define the URL
url = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=14450;type=tournament'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all the match summary links
match_summary_links = []
all_rows = soup.select('table.ds-table > tbody > tr')
for row in all_rows:
    link = "https://www.espncricinfo.com" + row.select('td')[6].find('a').get('href')
    match_summary_links.append(link)

# Iterate through each match summary link
batting_summary = []
for match_link in match_summary_links:
    # Send a GET request to the match summary link
    match_response = requests.get(match_link)
    match_soup = BeautifulSoup(match_response.text, 'html.parser')

    # Extract match details
    match_info = match_soup.find('span', text="Match Details").find_parent('div').find_next_sibling().text
    team1, team2 = match_info.split(' Vs ')

    # Extract batting details for the only innings available in the table
    innings_rows = match_soup.select('table.ds-table > tbody > tr')
    for index, row in enumerate(innings_rows):
        tds = row.select('td')
        batting_summary.append({
            "match": match_info,
            "teamInnings": team1 if index == 0 else team2,
            "battingPos": index + 1,
            "batsmanName": tds[0].text.strip(),
            "dismissal": tds[1].text.strip(),
            "runs": tds[2].text.strip(),
            "balls": tds[3].text.strip(),
            "4s": tds[4].text.strip(),
            "6s": tds[5].text.strip(),
            "SR": tds[6].text.strip()
        })

# Print the batting summary
for player in batting_summary:
    print(player)


In [8]:
batting_summary

[]

In [45]:
from IPython.display import display, Javascript

# Define your JavaScript code as a string
javascript_code = """
navigate('https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=14450;type=tournament');

let links = parse().matchSummaryLinks;
for(let i of links) { 
  next_stage({url: i}) 
}


let links = []
const allRows = $('table.engineTable > tbody > tr.data1');
 allRows.each((index, element) => {
  const tds = $(element).find('td');
  const rowURL = "https://www.espncricinfo.com" +$(tds[6]).find('a').attr('href');
  links.push(rowURL);
 })
return {
  'matchSummaryLinks': links
};




navigate(input.url);
collect(parse());


var match = $('div').filter(function(){
	return $(this)
      .find('span > span > span').text() === String("Match Details") 
}).siblings()
team1 = $(match.eq(0)).find('span > span > span').text().replace(" Innings", "")
team2 = $(match.eq(1)).find('span > span > span').text().replace(" Innings", "")
matchInfo = team1 +  ' Vs ' + team2

var tables = $('div > table.ci-scorecard-table');
var firstInningRows = $(tables.eq(0)).find('tbody > tr').filter(function(index, element){
  return $(this).find("td").length >= 8
})

var secondInningsRows = $(tables.eq(1)).find('tbody > tr').filter(function(index, element){
  return $(this).find("td").length >= 8
});


var battingSummary = []
firstInningRows.each((index, element) => {
  var tds = $(element).find('td');
  battingSummary.push({
  		"match": matchInfo,
  		"teamInnings": team1,
   		"battingPos": index+1,
  		"batsmanName": $(tds.eq(0)).find('a > span > span').text().replace(' ', ''),
    	"dismissal": $(tds.eq(1)).find('span > span').text(),
  		"runs": $(tds.eq(2)).find('strong').text(), 
  		"balls": $(tds.eq(3)).text(),
  		"4s": $(tds.eq(5)).text(),
  		"6s": $(tds.eq(6)).text(),
 		"SR": $(tds.eq(7)).text()
  });
});

secondInningsRows.each((index, element) => {
  var tds = $(element).find('td');
   battingSummary.push({
  		"match": matchInfo,
  		"teamInnings": team2,
   		"battingPos": index+1,
  		"batsmanName": $(tds.eq(0)).find('a > span > span').text().replace(' ', ''),
     	"dismissal": $(tds.eq(1)).find('span > span').text(),
  		"runs": $(tds.eq(2)).find('strong').text(), 
  		"balls": $(tds.eq(3)).text(),
  		"4s": $(tds.eq(5)).text(),
  		"6s": $(tds.eq(6)).text(),
 		"SR": $(tds.eq(7)).text()
  });
});

return {"battingSummary": battingSummary}
"""
# Execute the JavaScript code
result = Javascript(javascript_code)

# Display the result
display(result)

<IPython.core.display.Javascript object>